In [47]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from skimage.io import imread
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Conv2D, Dropout, MaxPooling2D, Conv2DTranspose, concatenate
from keras.models import Model
from keras import backend as K
from keras.losses import binary_crossentropy
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau

In [48]:
# Constants for data preparation and augmentation
SAMPLES_PER_GROUP = 4000  # Number of samples per group for dataset balancing
ALPHA = 0.8               # Alpha parameter for loss function
GAMMA = 2                 # Gamma parameter for loss function
VALID_IMG_COUNT = 600     # Number of images for validation
IMG_SCALING = (3, 3)      # Downsampling factor for image preprocessing
AUGMENT_BRIGHTNESS = False  # Flag to enable brightness augmentation (True/False)

In [44]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
# Navigate to the dataset folder
! cd /content/drive/My\ Drive/DataSet

In [4]:
# Upload Kaggle API key
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ivanyutaolexandr","key":"72680c1c371f2ee2f7131ebdb8199b9e"}'}

In [5]:
# Set up Kaggle API credentials
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!rm kaggle.json

In [6]:
# Download dataset from Kaggle
!kaggle competitions download -c airbus-ship-detection

100% 28.6G/28.6G [05:04<00:00, 166MB/s]
100% 28.6G/28.6G [05:04<00:00, 101MB/s]


In [7]:
# Extract the dataset
from zipfile import ZipFile
file_name="/content/airbus-ship-detection.zip"

with ZipFile(file_name,'r') as zip_file_:
  zip_file_.extractall()

Done


In [49]:
base_dir = '/content'

train_dir = base_dir + '/train_v2/'
test_dir = base_dir + '/test_v2/'

In [51]:
train = os.listdir(train_dir)
test = os.listdir(test_dir)

In [52]:
# Data Preparation
def prepare_data(data_path):
    masks = pd.read_csv(os.path.join(data_path, 'train_ship_segmentations_v2.csv'))
    not_empty = pd.notna(masks.EncodedPixels)

    print(not_empty.sum(), 'masks in', masks[not_empty].ImageId.nunique(), 'images')
    print((~not_empty).sum(), 'empty images in', masks.ImageId.nunique(), 'total images')

    masks['ships'] = masks['EncodedPixels'].map(lambda c_row: 1 if isinstance(c_row, str) else 0)
    unique_img_ids = masks.groupby('ImageId').agg({'ships': 'sum'}).reset_index()
    unique_img_ids['has_ship'] = unique_img_ids['ships'].map(lambda x: 1.0 if x > 0 else 0.0)

    masks.drop(['ships'], axis=1, inplace=True)

    return masks, unique_img_ids

In [53]:
masks, unique_img_ids = prepare_data(base_dir)

81723 masks in 42556 images
150000 empty images in 192556 total images


In [54]:
# Let's balance the dataset by randomly selecting SAMPLES_PER_GROUP elements
# for each category (groups based on the number of ships) to avoid imbalance
# between categories.

balanced_train_df = unique_img_ids.groupby('ships').apply(lambda x: x.sample(SAMPLES_PER_GROUP) if len(x) > SAMPLES_PER_GROUP else x)

In [55]:
# Function for decoding the RLE representation of a mask in an image.
def decode_rle(encoded_mask, image_shape=(768, 768)):
    components = encoded_mask.split()
    start_indices, lengths = [np.asarray(x, dtype=int) for x in (components[0:][::2], components[1:][::2])]
    start_indices -= 1
    end_indices = start_indices + lengths
    decoded_mask = np.zeros(image_shape[0] * image_shape[1], dtype=np.uint8)

    for start, end in zip(start_indices, end_indices):
        decoded_mask[start:end] = 1

    return decoded_mask.reshape(image_shape).T

In [56]:
# Function to combine individual ship masks into a single array
def generate_single_mask(mask_list):
    all_masks_array = np.zeros((768, 768), dtype=np.uint8)

    for individual_mask in mask_list:
        if isinstance(individual_mask, str):
            all_masks_array |= decode_rle(individual_mask)

    return all_masks_array

In [57]:
# Downsampling in preprocessing
def make_image_gen(in_df, batch_size = 48):
    all_batches = list(in_df.groupby('ImageId'))
    out_rgb = []
    out_mask = []
    batch_counter = 0
    while True:
        np.random.shuffle(all_batches)
        for c_img_id, c_masks in all_batches:
            rgb_path = os.path.join(train_dir, c_img_id)
            c_img = imread(rgb_path)
            c_mask = np.expand_dims(generate_single_mask(c_masks['EncodedPixels'].values), -1)

            # Output image and mask shapes
            #print(f"Batch #{batch_counter}, Image shape: {c_img.shape}, Mask shape: {c_mask.shape}")

            if IMG_SCALING is not None:
                c_img = c_img[::IMG_SCALING[0], ::IMG_SCALING[1]]
                c_mask = c_mask[::IMG_SCALING[0], ::IMG_SCALING[1]]
            out_rgb += [c_img]
            out_mask += [c_mask]
            if len(out_rgb)>=batch_size:
                yield np.stack(out_rgb, 0)/255.0, np.stack(out_mask, 0).astype(np.float32)
                out_rgb, out_mask=[], []

In [58]:
# Split dataset into training and validation sets
train_ids, valid_ids = train_test_split(balanced_train_df, test_size = 0.2, stratify = balanced_train_df['ships'])
train_df = pd.merge(masks, train_ids)
valid_df = pd.merge(masks, valid_ids)
print(train_df.shape[0], 'training masks')
print(valid_df.shape[0], 'validation masks')


train_gen = make_image_gen(train_df)
train_x, train_y = next(train_gen)
valid_x, valid_y = next(make_image_gen(valid_df, VALID_IMG_COUNT))

44212 training masks
11059 validation masks


In [59]:
# A generator function for augmenting images and their corresponding masks.
def create_aug_gen(in_gen, seed = None):
    np.random.seed(seed if seed is not None else np.random.choice(range(9999)))
    for in_x, in_y in in_gen:
        seed = np.random.choice(range(1234))
        # keep the seeds syncronized otherwise the augmentation to the images is different from the masks
        g_x = image_gen.flow(255*in_x,
                             batch_size = in_x.shape[0],
                             seed = seed,
                             shuffle=True)
        g_y = label_gen.flow(in_y,
                             batch_size = in_x.shape[0],
                             seed = seed,
                             shuffle=True)

        yield next(g_x)/255.0, next(g_y)

In [60]:
# Data Augmentation
args = dict(featurewise_center = False,
                  samplewise_center = False,
                  rotation_range = 45,
                  width_shift_range = 0.1,
                  height_shift_range = 0.1,
                  shear_range = 0.01,
                  zoom_range = [0.9, 1.25],
                  horizontal_flip = True,
                  vertical_flip = True,
                  fill_mode = 'reflect',
                  data_format = 'channels_last')

if AUGMENT_BRIGHTNESS:
    args[' brightness_range'] = [0.5, 1.5]
image_gen = ImageDataGenerator(**args)

if AUGMENT_BRIGHTNESS:
    args.pop('brightness_range')
label_gen = ImageDataGenerator(**args)

cur_gen = create_aug_gen(train_gen)
t_x, t_y = next(cur_gen)

import gc; gc.enable()
gc.collect();

In [61]:
# Creates a U-Net model for image semantic segmentation.
def create_unet(filters=8, img_size=(256, 256, 3), dropout_rate=0.1, kernel_size=(3, 3), pool_size=(2, 2), strides=(2, 2)):
    """
    Args:
        filters (int, optional): The number of filters in the first layer, 8 by default.
        img_size (tuple, optional): The input image size, (256, 256, 3) by default.
        dropout_rate (float, optional): The dropout rate, 0.1 by default.
        kernel_size (tuple, optional): The kernel size for convolutional layers, (3, 3) by default.
        pool_size (tuple, optional): The pool size for max pooling layers, (2, 2) by default.
        strides (tuple, optional): The strides for transpose convolutional layers, (2, 2) by default.

    Returns:
        keras.models.Model: The implemented U-Net model.
    """
    inputs = Input(img_size)

    # Contraction path
    c1 = Conv2D(filters, kernel_size, activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
    c1 = Dropout(dropout_rate)(c1)
    c1 = Conv2D(filters, kernel_size, activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D(pool_size)(c1)

    c2 = Conv2D(filters * 2, kernel_size, activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(dropout_rate)(c2)
    c2 = Conv2D(filters * 2, kernel_size, activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D(pool_size)(c2)

    c3 = Conv2D(filters * 4, kernel_size, activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(dropout_rate)(c3)
    c3 = Conv2D(filters * 4, kernel_size, activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = MaxPooling2D(pool_size)(c3)

    c4 = Conv2D(filters * 8, kernel_size, activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Dropout(dropout_rate)(c4)
    c4 = Conv2D(filters * 8, kernel_size, activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = MaxPooling2D(pool_size)(c4)

    # Bridge (1024)
    c5 = Conv2D(filters * 16, kernel_size, activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = Dropout(dropout_rate)(c5)
    c5 = Conv2D(filters * 16, kernel_size, activation='relu', kernel_initializer='he_normal', padding='same')(c5)

    # Expansive path
    u6 = Conv2DTranspose(filters * 8, pool_size, strides=strides, padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(filters * 8, kernel_size, activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = Dropout(dropout_rate)(c6)
    c6 = Conv2D(filters * 8, kernel_size, activation='relu', kernel_initializer='he_normal', padding='same')(c6)

    u7 = Conv2DTranspose(filters * 4, pool_size, strides=strides, padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(filters * 4, kernel_size, activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Dropout(dropout_rate)(c7)
    c7 = Conv2D(filters * 4, kernel_size, activation='relu', kernel_initializer='he_normal', padding='same')(c7)

    u8 = Conv2DTranspose(filters * 2, pool_size, strides=strides, padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(filters * 2, kernel_size, activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Dropout(dropout_rate)(c8)
    c8 = Conv2D(filters * 2, kernel_size, activation='relu', kernel_initializer='he_normal', padding='same')(c8)

    u9 = Conv2DTranspose(filters, pool_size, strides=strides, padding='same')(c8)
    u9 = concatenate([u9, c1])
    c9 = Conv2D(filters, kernel_size, activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Dropout(dropout_rate)(c9)
    c9 = Conv2D(filters, kernel_size, activation='relu', kernel_initializer='he_normal', padding='same')(c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid', padding='same')(c9)

    model = Model(inputs=[inputs], outputs=[outputs], name='unet')
    print(model.summary())

    return model

In [62]:
model = create_unet()

Model: "unet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d_19 (Conv2D)          (None, 256, 256, 8)          224       ['input_2[0][0]']             
                                                                                                  
 dropout_9 (Dropout)         (None, 256, 256, 8)          0         ['conv2d_19[0][0]']           
                                                                                                  
 conv2d_20 (Conv2D)          (None, 256, 256, 8)          584       ['dropout_9[0][0]']           
                                                                                               

In [63]:
# Callbacks setting
weight_path="models/{}_weights.hdf5".format('model')

checkpoint = ModelCheckpoint(
    weight_path,
    monitor='val_dice_coef', 
    verbose=1,
    save_best_only=True, 
    mode='max', 
    save_weights_only = True
)

early = EarlyStopping(
    monitor="val_dice_coef",
    mode="max",
    patience=15
)

reduceLROnPlat = ReduceLROnPlateau(
    monitor='val_dice_coef', 
    factor=0.5,
    patience=3,
    verbose=1, 
    mode='max', 
    epsilon=0.0001, 
    cooldown=2, 
    min_lr=1e-6
)


callbacks_list = [checkpoint, early, reduceLROnPlat]

In [64]:
# Custom Loss Function
def Loss(targets, inputs, alpha=ALPHA, gamma=GAMMA):
    inputs = K.flatten(inputs)
    targets = K.flatten(targets)

    BCE = K.binary_crossentropy(targets, inputs)
    BCE_EXP = K.exp(-BCE)
    loss = K.mean(alpha * K.pow((1 - BCE_EXP), gamma) * BCE)

    return loss

In [71]:
# Dice Coefficient Metric
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

In [72]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-3), loss=Loss, metrics=[dice_coef, 'binary_accuracy'])

step_count = min(5, train_df.shape[0]//48)
aug_gen = create_aug_gen(make_image_gen(train_df))

model.fit(
    aug_gen, 
    steps_per_epoch=step_count, 
    epochs=15, 
    validation_data=(valid_x, valid_y), 
    callbacks=callbacks_list, 
    workers=1 
)

Epoch 1/15
5/5 [==============================] - ETA: 0s - loss: 0.0213 - dice_coef: 0.0045 - binary_accuracy: 0.9935 
Epoch 1: val_dice_coef did not improve from 0.00510
5/5 [==============================] - 179s 38s/step - loss: 0.0213 - dice_coef: 0.0045 - binary_accuracy: 0.9935 - val_loss: 0.0170 - val_dice_coef: 0.0035 - val_binary_accuracy: 0.9952 - lr: 0.0010
Epoch 2/15
5/5 [==============================] - ETA: 0s - loss: 0.0195 - dice_coef: 0.0048 - binary_accuracy: 0.9936 
Epoch 2: val_dice_coef improved from 0.00510 to 0.00516, saving model to models/model_weights.hdf5
5/5 [==============================] - 119s 26s/step - loss: 0.0195 - dice_coef: 0.0048 - binary_accuracy: 0.9936 - val_loss: 0.0161 - val_dice_coef: 0.0052 - val_binary_accuracy: 0.9952 - lr: 0.0010
Epoch 3/15
5/5 [==============================] - ETA: 0s - loss: 0.0176 - dice_coef: 0.0066 - binary_accuracy: 0.9930 
Epoch 3: val_dice_coef did not improve from 0.00516
5/5 [==============================] 

In [73]:
model_v2 = model
model_v2.save('models/model_full.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
